In [1]:
import datetime
import os
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import shutil
from shapely import Polygon

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SHConfig,
)
import geopandas as gpd

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image

c:\Users\Admin\Documents\GitHub\runaway-satellite-detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
jsons = [{"type":"Polygon","coordinates":[[[45.098534,41.234446],[45.098534,41.335318],[45.351906,41.335318],[45.351906,41.234446],[45.098534,41.234446]]]},
         {"type":"Polygon","coordinates":[[[45.346069,41.176328],[45.346069,41.263356],[45.564423,41.263356],[45.564423,41.176328],[45.346069,41.176328]]]},
         {"type":"Polygon","coordinates":[[[45.534554,41.065633],[45.534554,41.184597],[45.806122,41.184597],[45.806122,41.065633],[45.534554,41.065633]]]},
         ]
polygons = [Polygon(json['coordinates'][0]) for json in jsons]

In [3]:
start = datetime.datetime(2023, 3, 1)
end = datetime.datetime(2023, 6, 30)
n_chunks = 21
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

In [4]:
resolution = 10
config = SHConfig()

In [5]:
def get_image(path, bbox, time):
    size = bbox_to_dimensions(bbox, resolution=resolution)
    evalscript_true_color = """
        //VERSION=3

        function setup() {
            return {
                input: [{
                    bands: ["B02", "B03", "B04"]
                }],
                output: [
                {
                    id: "image",
                    bands: 3,
                    sampleType: SampleType.AUTO,
                }]
            };
        }

        function evaluatePixel(sample) {
            return {
                image: [4*sample.B04, 4*sample.B03, 4*sample.B02],
            };
        }
    """

    return SentinelHubRequest(
        data_folder = os.path.join(path, time[0] + '_' + time[1]),
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=time,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[
                   SentinelHubRequest.output_response("image", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        config=config,
    )

In [6]:
def extract_and_remove_tar_files(parent_directory):
    # Iterate through all directories in the specified parent directory
    for root, dirs, files in os.walk(parent_directory):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)

            # Get the list of files and directories inside the subdirectory
            sub_items = os.listdir(dir_path)
            
            inner_dir = os.path.join(dir_path, sub_items[0])
            
            if os.path.isdir(inner_dir):
                tar_file = os.path.join(inner_dir, "response.tar")
                
                if os.path.isfile(tar_file):
                    try:
                        with tarfile.open(tar_file, "r") as tar:
                            tar.extractall(path=dir_path)

                        os.remove(tar_file)
                        shutil.rmtree(inner_dir)
                    except Exception as e:
                        print(f"Error processing {tar_file}: {e}")

In [7]:
for polygon in polygons:
    bbox = BBox(polygon.bounds, CRS('EPSG:4326'))

    path = os.path.join('./azercosmos', polygon.wkt)
    os.makedirs(path, exist_ok=True)

    list_of_requests = []
    for slot in slots:
        list_of_requests.append(get_image(path, bbox, slot))
    list_of_requests = [request.download_list[0] for request in list_of_requests]

    data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=20, show_progress=True)

    extract_and_remove_tar_files(path)

100%|██████████| 20/20 [00:21<00:00,  1.10s/it]


In [9]:
for polygon in os.listdir('./azercosmos'):
    path = os.path.join('./azercosmos', polygon)
    for date in os.listdir(path):
        date_path = os.path.join(path, date)
        for subdirectory in os.listdir(date_path):
            subdirectory_path = os.path.join(date_path, subdirectory)
            for file in os.listdir(subdirectory_path):
                file_path = os.path.join(subdirectory_path, file)
                shutil.move(file_path, date_path)
            os.rmdir(subdirectory_path)
                